In [ ]:
# Setting options for the plots
%matplotlib inline
%config InlineBackend.figure_formats={'retina', 'svg'}
%config InlineBackend.rc={'savefig.dpi': 150}

# Summary Report 

In [ ]:
import itertools
import json
import os
import re
import pickle
import platform
import time

from collections import defaultdict as dd
from functools import partial
from os.path import abspath, dirname, exists, join
from string import Template

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from matplotlib import pyplot as plt

from IPython import sys_info
from IPython.display import display, HTML, Image, Javascript, Markdown, SVG

from rsmtool.utils import (float_format_func,
                           get_output_directory_extension,
                           int_or_float_format_func,
                           bold_highlighter,
                           color_highlighter,
                           show_thumbnail)

from rsmtool.reader import DataReader
from rsmtool.writer import DataWriter
from rsmtool.version import VERSION as rsmtool_version

<style type="text/css">
  div.prompt.output_prompt { 
    color: white; 
  }
  
  span.highlight_color {
    color: red;
  }
  
  span.highlight_bold {
    font-weight: bold;  
  }
    
  @media print {
    @page {
      size: landscape;
      margin: 0cm 0cm 0cm 0cm;
    }

    * {
      margin: 0px;
      padding: 0px;
    }

    #toc {
      display: none;
    }

    span.highlight_color, span.highlight_bold {
        font-weight: bolder;
        text-decoration: underline;
    }

    div.prompt.output_prompt {
      display: none;
    }
    
    h3#Python-packages, div#packages {
      display: none;
  }
</style>

In [ ]:
# NOTE: you will need to set the following manually
# if you are using this notebook interactively.
summary_id = os.environ.get('SUMMARY_ID')
description = os.environ.get('DESCRIPTION')
jsons_string = os.environ.get('JSONS')
jsons = jsons_string.split('%%')
output_dir = os.environ.get('OUTPUT_DIR')
use_thumbnails = os.environ.get('USE_THUMBNAILS') == '1'
file_format_summarize = os.environ.get('FILE_FORMAT')

# groups for analysis by prompt or subgroup.
# set to 'prompt' for the standard analysis of 'prompt%%subgroup1%%subgroup2' for subgroup analysis.
groups_desc_string = os.environ.get('GROUPS_FOR_DESCRIPTIVES') 
groups_desc = groups_desc_string.split('%%')
groups_eval_string = os.environ.get('GROUPS_FOR_EVALUATIONS') 
groups_eval = groups_eval_string.split('%%')
# javascript path
javascript_path = os.environ.get("JAVASCRIPT_PATH")

In [ ]:
# initialize id generator for thumbnails
id_generator = itertools.count(1)

In [ ]:
with open(join(javascript_path, "sort.js"), "r", encoding="utf-8") as sortf:
    display(Javascript(data=sortf.read()))

In [ ]:
# load the information about all models
model_list = []
for json_file in jsons:
    model_config = json.load(open(json_file))
    model_id = model_config['experiment_id']
    model_csvdir = dirname(json_file)
    model_file_format = get_output_directory_extension(model_csvdir, model_id)
    model_list.append((model_id, model_config, model_csvdir, model_file_format))

# create a list of 

In [ ]:
Markdown("This report presents the analysis for **{}**: {} \n ".format(summary_id, description))


In [ ]:
HTML(time.strftime('%c'))

In [ ]:
# get a matched list of model ids and descriptions
models_and_desc = zip([model_id for (model_id, config, csvdir, model_file_format) in model_list],
                      [config['description'] for (model_id, config, csvdir, file_format) in model_list])
model_desc_list = '\n\n'.join(['**{}**: {}'.format(m, d) for (m, d) in models_and_desc])

Markdown("The report compares the following models: \n\n {}".format(model_desc_list))

In [ ]:
if use_thumbnails:
    display(Markdown("""***Note: Images in this report have been converted to """
                     """clickable thumbnails***"""))

In [ ]:
%%html
<div id="toc"></div>